## 1. Setup and Imports

This script uses libraries for cloud data access (google.cloud.bigquery), process mining and visualization (pm4py), environment management (dotenv), data handling (pandas), and language model orchestration (langchain and langchain_openai).

In [2]:
# Import libraries for cloud data access
from google.cloud import bigquery
from google_auth_oauthlib.flow import InstalledAppFlow

In [3]:
# Import libraries for logging
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:
# Import libraries for data handling
import pandas as pd
import yaml

In [5]:
# Import libraries for process mining
import pm4py

In [6]:
# Import libraries for RAG (Retrieval-Augmented Generation)
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.chat_models import init_chat_model
from langchain.schema.document import Document

In [7]:
# Import libraries for loading environment variables
import os
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
# Import libraries for parsing XML files into TXT files
import xml.etree.ElementTree as ET

## 2. Authentication Setup

Before accessing Google Cloud services, a service account must be created in the Google Cloud Console, with the necessary IAM roles (e.g., BigQuery Admin) assigned to it. The service account’s JSON key file is securely stored locally, and its path is set using the GOOGLE_APPLICATION_CREDENTIALS environment variable to enable programmatic authentication.

In [8]:
# Configuration
project_id = "integration-of-pm-and-llms"
client_secret_path = "/Users/alejandromateocobo/Documents/PythonProjects/Integration_Of_LLMs_And_Process_Mining/keys/client_secret_316641064865-57id3o26obibotvs226jeevisjdujha5.apps.googleusercontent.com.json"

# Authentication flow
logging.info("Starting OAuth authentication...")
SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]
flow = InstalledAppFlow.from_client_secrets_file(
    client_secret_path,
    scopes=SCOPES
)
credentials = flow.run_local_server(port=0)
client = bigquery.Client(credentials=credentials, project=project_id)

logging.info("Authentication successful.")
logging.info("BigQuery client initialized.")

2025-07-16 12:33:51,247 - INFO - Starting OAuth authentication...


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=316641064865-57id3o26obibotvs226jeevisjdujha5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51627%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform&state=gVeddhRU61VQB3aXNUAtLOzNY8QCzt&access_type=offline


2025-07-16 12:34:01,471 - INFO - "GET /?state=gVeddhRU61VQB3aXNUAtLOzNY8QCzt&code=4/0AVMBsJhEfPIjP6oyrRxHZb8bUFVTP1yVAmPIdLL3s_q57kmWFUJdb17DGoOxywlFK-a-Tw&scope=https://www.googleapis.com/auth/cloud-platform HTTP/1.1" 200 65
2025-07-16 12:34:01,859 - INFO - Authentication successful.
2025-07-16 12:34:01,863 - INFO - BigQuery client initialized.


## 3. Query BigQuery

Google Cloud Platform (GCP) is a suite of cloud computing services that enables scalable storage, processing, and data analysis using Google’s infrastructure. To use GCP for analyzing clinical datasets like MIMIC-III, users must create a Google account, set up a GCP project with billing, enable the necessary APIs, and configure OAuth client authentication to securely access cloud resources. The MIMIC-III database, which contains detailed health records from over 40,000 critical care patients, can be accessed through Google BigQuery for efficient cloud-based analysis, which is the recommended method by the MIT Lab for Computational Physiology.

In [9]:
query = """
        SELECT e.*
        FROM `integration-of-pm-and-llms.integration_of_pm_and_llms.filtered_eventlog` e
        INNER JOIN `physionet-data.mimiciii_clinical.icustays` icu
        ON e.icustay_id = icu.icustay_id
        WHERE e.icustay_id IN (211555, 290738, 236225, 213113)
        AND e.linksto = 'datetimeevents'
    """

# Check if the client is initialized before running the query
if not client:
    raise Exception("BigQuery client not initialized. Run authenticate() first.")

# Run the query
logging.info("Running user-provided query...")
job = client.query(query)

logging.info("Waiting for query to complete...")
df = job.to_dataframe(progress_bar_type='tqdm')

logging.info(f"Query completed. Retrieved {len(df)} rows.")

2025-07-16 12:34:08,883 - INFO - Running user-provided query...


2025-07-16 12:34:10,816 - INFO - Waiting for query to complete...


Job ID be94b387-ee38-4e62-8325-6e28f34a51cd successfully executed: 100%|██████████|
Downloading: 100%|██████████|

2025-07-16 12:34:13,103 - INFO - Query completed. Retrieved 1616 rows.


In [10]:
# Process the dataframe as needed
print(df.head())

   itemid  subject_id  hadm_id  icustay_id     event_timestamp  \
0  226724       23657   118450      290738 2146-07-29 00:00:00   
1  225353       97599   135263      213113 2142-01-01 14:22:00   
2  224280       97599   135263      213113 2142-01-19 00:00:00   
3  224295       53232   115492      211555 2151-06-29 12:38:00   
4  225351       97599   135263      213113 2142-01-06 00:00:00   

                               label                 category         linksto  
0                Hospital Admit Date                      ADT  datetimeevents  
1        PA Catheter Dressing Change  Access Lines - Invasive  datetimeevents  
2         Multi Lumen Insertion Date  Access Lines - Invasive  datetimeevents  
3  Cordis/Introducer Dressing Change  Access Lines - Invasive  datetimeevents  
4             PA Catheter Cap Change  Access Lines - Invasive  datetimeevents  


## 4. Prepare Event Log for PM4PY

In this step, the dataset is reformatted to match the structure expected by PM4PY, where each event log requires a case identifier, an activity name, and a timestamp. The data is then converted into a PM4PY event log object, which enables process mining algorithms to analyze the sequence of events across cases.

In [12]:
# Rename columns for PM4PY (using icustay_id as case identifier)
df_eventlog = df.rename(columns={
    "icustay_id": "case:concept:name",  # Update to use icustay_id as case identifier
    "event_timestamp": "time:timestamp",
    "label": "concept:name"
})

# Convert the timestamp column to datetime
df_eventlog["time:timestamp"] = pd.to_datetime(df_eventlog["time:timestamp"], errors="coerce")

# Format the dataframe using pm4py.utils.format_dataframe
df_eventlog = pm4py.utils.format_dataframe(
    df_eventlog,
    case_id='case:concept:name',
    activity_key='concept:name',
    timestamp_key='time:timestamp'
)

# Convert the dataframe to an event log object
event_log = pm4py.convert_to_event_log(df_eventlog)

# Print basic statistics
logging.info(f"Total cases: {len(set(df_eventlog['case:concept:name']))}")
logging.info(f"Total activities: {len(set(df_eventlog['concept:name']))}")
logging.info(f"Total events: {len(df_eventlog)}")

2025-07-16 12:34:22,765 - INFO - Total cases: 4
2025-07-16 12:34:22,768 - INFO - Total activities: 35
2025-07-16 12:34:22,769 - INFO - Total events: 1616


In [13]:
# Preview the formatted event log
print(df_eventlog.head())

   itemid  subject_id  hadm_id case:concept:name            time:timestamp  \
0  226515       53232   115492            211555 2128-01-02 00:00:00+00:00   
1  225354       53232   115492            211555 2151-06-29 00:00:00+00:00   
2  224296       53232   115492            211555 2151-06-29 00:00:00+00:00   
3  225755       53232   115492            211555 2151-06-29 00:00:00+00:00   
4  224298       53232   115492            211555 2151-06-29 00:00:00+00:00   

                       concept:name                   category  \
0                     Date of Birth                        ADT   
1        PA Catheter Insertion Date    Access Lines - Invasive   
2  Cordis/Introducer Insertion Date    Access Lines - Invasive   
3           18 Gauge Insertion Date  Access Lines - Peripheral   
4   Cordis/Introducer Tubing Change    Access Lines - Invasive   

          linksto  @@index  @@case_index  
0  datetimeevents        0             0  
1  datetimeevents        1             0  
2  da

## 5. Discover and Visualize the Directly-Follows Graph (DFG) with PM4PY

In this step, the frequency-based Directly-Follows Graph (DFG) is discovered from the event log using PM4PY and visualized to understand the control-flow structure of the process. The DFG is then converted into a readable textual format to prepare it for further analysis with a Large Language Model (LLM).

In [14]:
# Discover the Directly-Follows Graph (DFG)
dfg, start_activities, end_activities = pm4py.discovery.discover_dfg(
    event_log,
    activity_key='concept:name',
    case_id_key='case:concept:name',
    timestamp_key='time:timestamp'
)

In [ ]:
# Visualize the DFG
pm4py.vis.view_dfg(
    dfg,
    start_activities,
    end_activities,
    format='png',       # You can also use 'svg' if supported
    bgcolor='white',
    rankdir='LR'        # Left-to-right layout
)

In [15]:
# Discover Petri net using the Inductive Miner
net, im, fm = pm4py.discovery.discover_petri_net_inductive(
    event_log,
    activity_key='concept:name',
    case_id_key='case:concept:name',
    timestamp_key='time:timestamp',
    multi_processing=True
)

In [ ]:
# Visualize the Petri net
pm4py.vis.view_petri_net(net, im, fm, format='png')

## 6. Database Schema

In [18]:
xml_path = "/Users/alejandromateocobo/Documents/PythonProjects/Integration_Of_LLMs_And_Process_Mining/scripts/filtered_mimic.mimiciii.xml"
output_path = "/Users/alejandromateocobo/Documents/PythonProjects/Integration_Of_LLMs_And_Process_Mining/context/about/schema.txt"

logging.info("Parsing XML schema to generate textual abstraction...")

# Load and sanitize XML content to avoid parse errors
with open(xml_path, "r", encoding="utf-8") as file:
    xml_content = file.read()

# Escape problematic XML tokens
xml_content = xml_content.replace("<=", "&lt;=").replace(">=", "&gt;=")

# Parse XML from sanitized string
root = ET.fromstring(xml_content)

tables_info = {}
relations = []

for table in root.findall(".//table"):
    table_name = table.get("name")
    columns = []

    for column in table.findall("column"):
        col_name = column.get("name")
        col_type = column.get("type")
        col_remark = column.get("remarks", "").strip()
        columns.append(f"- {col_name} ({col_type}): {col_remark}")

        # Foreign key (parent)
        parent = column.find("parent")
        if parent is not None:
            ref_table = parent.get("table")
            ref_column = parent.get("column")
            relations.append(f"{table_name}.{col_name} → {ref_table}.{ref_column}")

    tables_info[table_name] = columns

    # Foreign key (child)
    for column in table.findall("column"):
        for child in column.findall("child"):
            source_col = column.get("name")
            target_table = child.get("table")
            target_col = child.get("column")
            relations.append(f"{target_table}.{target_col} → {table_name}.{source_col}")

# Write abstraction to text file with metadata header
with open(output_path, "w", encoding="utf-8") as f:
    # Metadata header
    f.write("""---
type: about
tags: [mimic-iii, schema]
filename: schema
---
\n""")

    # Table definitions
    for table, columns in tables_info.items():
        f.write(f"## Table: {table}\n")
        for col in columns:
            f.write(col + "\n")
        f.write("\n")

    # Foreign key relations
    f.write("## Foreign Key Relations\n")
    for rel in sorted(set(relations)):
        f.write(f"- {rel}\n")

logging.info(f"Textual schema abstraction written to: {output_path}")

2025-07-16 12:37:14,871 - INFO - Parsing XML schema to generate textual abstraction...
2025-07-16 12:37:14,890 - INFO - Textual schema abstraction written to: /Users/alejandromateocobo/Documents/PythonProjects/Integration_Of_LLMs_And_Process_Mining/data/context/schema_abstraction.txt


In [19]:
# Print the contents of the schema abstraction text file
with open(output_path, "r", encoding="utf-8") as f:
    print(f.read())

---
type: semantic_model
tags: [mimic-iii, schema]
filename: schema_abstraction.txt
---

## Table: admissions
- row_id (int4): Unique row identifier.
- subject_id (int4): Foreign key. Identifies the patient.
- hadm_id (int4): Primary key. Identifies the hospital stay.
- admittime (timestamp): Time of admission to the hospital.
- dischtime (timestamp): Time of discharge from the hospital.
- deathtime (timestamp): Time of death.
- admission_type (varchar): Type of admission, for example emergency or elective.
- admission_location (varchar): Admission location.
- discharge_location (varchar): Discharge location
- insurance (varchar): Insurance type.
- language (varchar): Language.
- religion (varchar): Religon.
- marital_status (varchar): Marital status.
- ethnicity (varchar): Ethnicity.
- edregtime (timestamp): 
- edouttime (timestamp): 
- diagnosis (varchar): Diagnosis.
- hospital_expire_flag (int2): 
- has_chartevents_data (int2): Hospital admission has at least one observation in the 

## 7. LangChain

In [9]:
# Load Pinecone and OpenAI keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(name="integration-of-pm-and-llms")

In [10]:
# Delete all existing vectors in the index (WARNING: irreversible)
index.delete(delete_all=True)

{}

In [12]:
# Load all .txt files recursively from context/ and subfolders
loader = DirectoryLoader(
    path="/Users/alejandromateocobo/Documents/PythonProjects/Integration_Of_LLMs_And_Process_Mining/context",
    glob="**/*.txt",
    loader_cls=TextLoader,
    recursive=True  # Important if supported by your LangChain version
)
documents = loader.load()

In [14]:
def extract_metadata_from_text(text: str) -> (dict, str):
    if text.startswith("---"):
        parts = text.split("---", 2)
        if len(parts) >= 3:
            metadata = yaml.safe_load(parts[1])
            content = parts[2].strip()
            return metadata, content
    return {}, text.strip()

def batched(iterable, n=10):
    """Yield successive n-sized chunks from iterable."""
    for i in range(0, len(iterable), n):
        yield iterable[i:i + n]

# Load and process documents
processed_docs = []
for doc in documents:
    metadata, content = extract_metadata_from_text(doc.page_content)
    processed_docs.append(Document(page_content=content, metadata=metadata))

# Split into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs = text_splitter.split_documents(processed_docs)

# Generate unique chunk IDs
ids = [f"doc_{i}" for i in range(len(chunked_docs))]

# Compute embeddings
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")

# Store in Pinecone with metadata
vector_store = PineconeVectorStore(index=index, embedding=embeddings)
for doc_batch, id_batch in zip(batched(chunked_docs, 100), batched(ids, 100)):
    vector_store.add_documents(documents=doc_batch, ids=id_batch)

logging.info("Documents with YAML metadata embedded and stored in Pinecone.")

2025-07-16 16:57:53,037 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 16:57:57,031 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 16:58:00,941 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 16:58:03,701 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 16:58:06,347 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 16:58:09,630 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 16:58:12,218 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 16:58:16,090 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 16:58:18,532 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 16:58:20,551 - INFO - HTTP

In [25]:
llm = init_chat_model("gpt-4o-mini", model_provider="openai")
query = "What is the most common workflow for patients in the ICU??"

retrieved_docs = vector_store.similarity_search(query, k=10)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

prompt = f"""You are a helpful and knowledgeable assistant specialized in process mining. Your role is to analyze process-related data, models, and documentation to provide accurate and concise answers.

Here’s a user question: {query}

Here is some extracted documentation and context:

{docs_content}

Now please answer the question as clearly and precisely as possible:
"""

print(prompt)
answer = llm.invoke(prompt)
print(answer)

2025-07-13 17:30:52,138 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


You are a helpful and knowledgeable assistant specialized in process mining. Your role is to analyze process-related data, models, and documentation to provide accurate and concise answers.

Here’s a user question: What is the most common workflow for patients in the ICU??

Here is some extracted documentation and context:

Abstract
MIMIC-III is a large, freely-available database comprising deidentified health-related data associated with over forty thousand patients who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012. The database includes information such as demographics, vital sign measurements made at the bedside (~1 data point per hour), laboratory test results, procedures, medications, caregiver notes, imaging reports, and mortality (including post-hospital discharge). MIMIC supports a diverse range of analytic studies spanning epidemiology, clinical decision-rule improvement, and electronic tool development. It is notable for three

2025-07-13 17:30:56,894 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


content='The most common workflow for patients in the ICU, based on the extracted documentation, involves a series of key activities primarily related to vascular access management and dressing changes. Here are the main activities identified in the ICU processes:\n\n1. **Arterial Line Management**:\n   - Frequent "Arterial Line Dressing Change" (98 occurrences) and "Arterial line Tubing Change" (49 occurrences).\n   - "Arterial Line Insertion Date" is a common precursor activity, occurring 39 times.\n\n2. **Multi Lumen Catheter Management**:\n   - "Multi Lumen Dressing Change" occurs 96 times, followed closely by "Multi Lumen Insertion Date" (60 occurrences) and "Multi Lumen Tubing Change" (noted multiple transitions).\n\n3. **Dressing Changes**:\n   - Dressing changes for various types of lines (like "Sheath," "Cordis/Introducer," and "IABP") are also integral to patient workflow, with multiple instances reflecting regular maintenance and care.\n\n4. **Continuous Care Activities**:\n